<a href="https://colab.research.google.com/github/anitabiju005/SENTIMENT-ANALYSIS-PIPELINE/blob/main/Content_Governance_LLM_based_Sentiment_ANITA_BIJU_S6_CSE_A_7012091669.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install libraries
!pip install --quiet transformers datasets keybert spacy contractions
!python -m spacy download en_core_web_sm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 138.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
#Import models and libraries
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import load_dataset, Dataset
import spacy
import contractions
from keybert import KeyBERT
from sklearn.model_selection import train_test_split

In [3]:
#Load Dataset
dataset = load_dataset("tweet_eval", "sentiment")
subset_size = 500
reviews = dataset["train"][:subset_size]["text"]
true_labels = dataset["train"][:subset_size]["label"]

# Map numeric labels to sentiment
label_map = {0:"NEGATIVE", 1:"NEUTRAL", 2:"POSITIVE"}
true_labels_mapped = [label_map[l] for l in true_labels]

# Train/Validation split
reviews_train, reviews_val, labels_train, labels_val = train_test_split(
    reviews, true_labels, test_size=0.2, random_state=42
)

print("Train samples:", len(reviews_train))
print("Validation samples:", len(reviews_val))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sentiment/train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

sentiment/test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

sentiment/validation-00000-of-00001.parq(…):   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Train samples: 400
Validation samples: 100


In [4]:
#Preprocessing Function
nlp = spacy.load("en_core_web_sm")
priority_words = ["don't", "no", "not", "never"]

def preprocess_text(text):
    # Expand contractions
    text = contractions.fix(text)
    text = text.lower()
    # Keep apostrophes, remove other special chars
    text = re.sub(r"[^a-zA-Z0-9\s']", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

#Remove proper nouns
def remove_proper_nouns(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if token.pos_ != "PROPN"])

#Divide text into chunks
def chunk_text(text, chunk_size=150):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [5]:
#Tokenization and Dataset Formatting
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create HuggingFace Datasets
train_dataset = Dataset.from_dict({"text": reviews_train, "labels": labels_train})
val_dataset   = Dataset.from_dict({"text": reviews_val,   "labels": labels_val})

# Tokenize
def tokenize_batch(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_batch, batched=True)
val_dataset   = val_dataset.map(tokenize_batch, batched=True)

# Set torch format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
#Fine Tuning
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anitabiju005 (anitabiju005-christ-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, langchain, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,0.697300
20,0.771800
30,0.599400
40,0.544700
50,0.533300
60,0.346000
70,0.346700
80,0.425200
90,0.344300
100,0.280300


TrainOutput(global_step=100, training_loss=0.488896815776825, metrics={'train_runtime': 84.2794, 'train_samples_per_second': 9.492, 'train_steps_per_second': 1.187, 'total_flos': 31655833070400.0, 'train_loss': 0.488896815776825, 'epoch': 2.0})

In [7]:
#Create pipeline
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=512
)

Device set to use cuda:0


In [8]:
#Structured Sentiment Summary
def structured_sentiment_summary(texts, top_n=3, chunk_size=150):
    if isinstance(texts, str):
        texts = [texts]

    kw_model = KeyBERT()
    results = []

    for t in texts:
        #Sentiment
        out = sentiment_pipeline(t)[0]
        label = out["label"].upper()        # POSITIVE / NEGATIVE / NEUTRAL
        conf  = round(out["score"], 3)

        #Preprocess + remove proper nouns
        clean_text = preprocess_text(t)
        clean_text = remove_proper_nouns(clean_text)

        #Chunk text for KeyBERT
        chunks = chunk_text(clean_text, chunk_size)

        #Extract key supporting phrases
        keywords = []
        for chunk in chunks:
            kws = kw_model.extract_keywords(
                chunk,
                keyphrase_ngram_range=(1,3),
                stop_words='english',
                top_n=5
            )
            for kw in kws:
                phrase = kw if isinstance(kw, str) else kw[0]
                keywords.append(phrase)

        # 4a️⃣ Add priority words if present
        for pw in priority_words:
            if pw in clean_text.split() and pw not in keywords:
                keywords.insert(0, pw)

        # 4b️⃣ Build unique list and ensure EXACT top_n phrases
        unique_keywords = []
        for kw in keywords:
            if kw not in unique_keywords:
                unique_keywords.append(kw)

        # Fallback: if fewer than top_n, fill from cleaned text
        if len(unique_keywords) < top_n:
            extra_words = clean_text.split()
            for w in extra_words:
                if w not in unique_keywords:
                    unique_keywords.append(w)
                if len(unique_keywords) >= top_n:
                    break

        # FINAL: exactly top_n supporting phrases
        top_phrases = unique_keywords[:top_n]

        #Follow-up suggestion
        follow_up = ""
        if label == "NEGATIVE":
            follow_up = "Consider rephrasing your review or providing constructive feedback."

        results.append({
            "text": t,
            "predicted_sentiment": label,
            "confidence": conf,
            "top_supporting_phrases": top_phrases,
            "follow_up_suggestions": follow_up
        })

    return pd.DataFrame(results)

In [9]:
#Test Summary Report
df_summary = structured_sentiment_summary(reviews_val)
df_summary

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,text,predicted_sentiment,confidence,top_supporting_phrases,follow_up_suggestions
0,Hillary gets emotional over talking about her ...,NEGATIVE,0.936,"[emotional talking mother, mother tomorrow fak...",Consider rephrasing your review or providing c...
1,My mom's over here tryna tell me that we're ta...,NEGATIVE,0.659,"[taking tomorrow nigga, month school slowyourr...",Consider rephrasing your review or providing c...
2,Caitlyn Jenner May Face Manslaughter Chargesk!...,NEUTRAL,0.662,"[jenner face manslaughter, manslaughter charge...",
3,"""Off to ER, for the Edinburgh Derby, c'mon the...",NEUTRAL,0.897,"[er hibs, hibs, er]",
4,"""J'regarde des infos sur Ragnarok (le film) &a...",NEUTRAL,0.859,"[not, thor make movie, movie loki]",
...,...,...,...,...,...
95,Any1 interested in @user tickets 4 the NIA on ...,NEUTRAL,0.894,"[user tickets nia, tickets nia, interested use...",
96,Q_Q Today was the day that Naruto shippuden: U...,NEGATIVE,0.900,"[today day shippuden, day shippuden suppose, d...",Consider rephrasing your review or providing c...
97,"""I'm going to Paramore at Rock Im Park in Nure...",NEUTRAL,0.543,"[paramore rock park, going paramore rock, para...",
98,"""Seth Rollins just out there stealing all of F...",NEUTRAL,0.711,"[rollins just stealing, rollins just, rollins]",


In [10]:
#Evaluating pipeline
pred_df = structured_sentiment_summary(reviews)
pred_labels = pred_df['predicted_sentiment'].tolist()  # string labels

# Compute metrics
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(true_labels_mapped, pred_labels)
f1 = f1_score(true_labels_mapped, pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.3f}")
print(f"Weighted F1-score: {f1:.3f}")


Accuracy: 0.886
Weighted F1-score: 0.885


In [16]:
#Generating numeric predictions for validation set
device = next(model.parameters()).device

preds = []

for batch in val_dataset:
    inputs = {
        "input_ids": batch["input_ids"].unsqueeze(0).to(device),
        "attention_mask": batch["attention_mask"].unsqueeze(0).to(device)
    }

    with torch.no_grad():
        output = model(**inputs)
        logits = output.logits
        pred = torch.argmax(logits, dim=1).item()
        preds.append(pred)

In [17]:
misclassified = []

for i in range(len(labels_val)):
    if preds[i] != labels_val[i]:
        misclassified.append((reviews_val[i][:300], labels_val[i], preds[i]))

print("\n--- Error Analysis (Sample 10 misclassifications) ---")
for text, true, pred in misclassified[:10]:
    print("\nTEXT:", text.replace("\n", " "))
    print("TRUE LABEL:", true, "| PRED:", pred)

print("\nTOTAL MISCLASSIFICATIONS =", len(misclassified), "samples")



--- Error Analysis (Sample 10 misclassifications) ---

TEXT: "Your 3rd gen. iPad with Retina display is worth a lot of money, especially at Amazon (MacDailyNews/MacDailyNews)
TRUE LABEL: 1 | PRED: 2

TEXT: New Jeremy Kyle starts Monday\u002c you bitch I\u2019m back to school then!!
TRUE LABEL: 1 | PRED: 0

TEXT: Check out Newsha Tavakolian at the Saatchi Gallery in London. Her photos of Iran may interest some of you!
TRUE LABEL: 1 | PRED: 2

TEXT: Sorry bout the stream last night I crashed out but will be on tonight for sure. Then back to Minecraft in pc tomorrow night.
TRUE LABEL: 1 | PRED: 0

TEXT: @user @user @user miss 11yo loves 13th street, BBC crime, etc, news hasn't had an impact yet - she still listens to me
TRUE LABEL: 1 | PRED: 2

TEXT: "Let this Christ, this King of Israel, come down now from the cross, that we may see and believe."
TRUE LABEL: 1 | PRED: 2

TEXT: Right guys\u002c last competition of the night... Like this status for a chance to win a copy of Judas Priest\u